In [1]:
import numpy as np
import itertools

# step1 计算sigma_i = 顶点i的 入度和 - 出度和
def sigma_cal(D1, n):
    sigma_list = [0] * n
    for ii in range(n):
        # 出度和
        d_out = len(np.where( D1[ii] < inf)[0])
        # 入度和
        d_in = 0
        for jj in range(n):
            if D1[jj][ii] < inf:
                d_in += 1
        sigma_ii = d_in - d_out
        sigma_list[ii] = sigma_ii
    return sigma_list

In [2]:
# step2 求出S，T，sigma， 求出D中最短路
def zuiduanlu(D1, n, sigma_list):
    S = []
    T = []
    for k in range(n):
        if sigma_list[k] > 0:
            S.append(k)
        elif sigma_list[k] < 0:
            T.append(k)

    Sigma = 0
    for v_i in S:
        Sigma += sigma_list[v_i]
    return S, T, Sigma

In [3]:
# 用Floyd算法,求出D中最短（v_s, v_t)路
def floyd(w, n, S, T):
    # 最短路的权矩阵u(n * n) 和各顶点的后进标号矩阵r(n * n)
    u = np.zeros((n, n))
    r = np.full((n, n), 0, dtype=int)

    for k in range(n + 1):
        # 初始化，弧数不多于1时，最短路即为两顶点的弧的权，从权矩阵中提取
        if k == 0:
            u = w
            # 记录第一条弧的头
            for i in range(n):
                for j in range(n):
                    r[i][j] = j + 1

        else:
            for i in range(n):
                for j in range(n):
                    if u[i][k - 1] + u[k - 1][i] < 0:
                        print('存在负回路！Floyd算法不适用！')
                        break
                    else:
                        if u[i][j] > u[i][k - 1] + u[k - 1][j]:
                            temp_r = r
                            r[i][j] = temp_r[i][k - 1]
                            temp_u = u
                            u[i][j] = temp_u[i][k - 1] + temp_u[k - 1][j]

    zuiduanlu_set = {}
    shortest_weight_set = {}
    # 正向追踪路径
    for i in S:
        for j in T:
            # 将起点加入路径
            vertex = [i]
            if i != j:
                temp = r[i][j]
                while temp != j + 1:
                    vertex.append(temp - 1)
                    temp = r[temp - 1][j]
                else:
                    vertex.append(temp - 1)
                w_sum = 0
            # 求每条最短路径的权和
            for kk in range(len(vertex) - 1):
                k1 = vertex[kk]
                k2 = vertex[kk + 1]
                w_sum += w[k1][k2]
            zuiduanlu_set[(i, j)] = vertex
            shortest_weight_set[(i, j)] = int(w_sum)
    return zuiduanlu_set, shortest_weight_set


In [4]:
# step3 构造赋权的完全二部图G
def bipartite_graph(shortest_weight_set, S, T, sigma_list):
    path = {}
    S_set = []
    T_set = []
    for value_s in S:
        for p in range(sigma_list[value_s]):
            S_set.append((value_s, p))
    for value_t in T:
        for q in range(-sigma_list[value_t]):
            T_set.append((value_t, q))
    for x_ip in S_set:
        for y_jq in T_set:
            path[(x_ip, y_jq)] = shortest_weight_set[(x_ip[0], y_jq[0])]
    return S_set, T_set, path

In [5]:
# stpe4 求最小权完美匹配M 共sigma的阶乘次
def find_match(S_set, T_set, G, Sigma):
    # 对S集做排列组合，共sigma的阶乘个组合
    S_list = list(itertools.permutations(S_set, Sigma))
    shortest_match_weight = 100
    for S_case in S_list:
        match_weight = 0
        for i in range(Sigma):
            match_weight += G[(S_case[i], T_set[i])]
        if match_weight < shortest_match_weight:
            shortest_match_weight = match_weight
            final_case = S_case
    M = []
    for j in range(Sigma):
        M.append((final_case[j][0], T_set[j][0]))
    return M


def euler_step1(v, v_1, v_2, w, D1):
    while v_2 != v:
        # 任取一条v_1的出弧，加入W中
        not_zero_set = np.where(D1[v_1] != 0)[0]
        v_2 = not_zero_set[0]
        w.append(v_2)
        D1[v_1][v_2] -= 1
        v_1 = v_2
    print(w)
    print(D1)
    return w, D1

In [6]:
# step5 欧拉回路求解
def euler(D1, v1):
    # 初始化，在Euler图中任取一个顶点v1
    v = v1
    v_1 = v1
    w = [v1]
    # 给v_2初始化一个取不到的值
    v_2 = len(D1)
    w, D1 = euler_step1(v, v_1, v_2, w, D1)
    # 当D不是空图时，在W中选择一个度数>0的顶点，把W改写为以该顶点为起点和终点的有向闭迹
    while np.where(D1 != 0)[0].size > 0:
        flag = 0
        for i in range(len(D1)):
            if np.where(D1[i] != 0)[0].size > 0 and i in w and flag == 0:
                index = w.index(i)
                w = w[index:-1] + w[:index]
                v = i
                v_1 = i
                v_2 = len(D1)
                euler_step1(v, v_1, v_2, w, D1)
                flag = 1
    return w


In [7]:
# 运行书上p208例子
if __name__ == '__main__':
    inf = float('inf')
    D = np.array([[inf, inf, inf, 4, inf],
                  [2, inf, 1, inf, inf],
                  [3, inf, inf, 6, inf],
                  [inf, inf, inf, inf, 2],
                  [3, 2, 5, inf, inf]])
    D_copy = D.copy()
    sigma_set = sigma_cal(D, 5)
    s, t, sigma = zuiduanlu(D, 5, sigma_set)
    zuiduanlu, shortest_weight = floyd(D, 5, s, t)
    s_set, t_set, graph = bipartite_graph(shortest_weight, s, t, sigma_set)
    m = find_match(s_set, t_set, graph, sigma)

    D_copy[D_copy < inf] = 1
    D_copy[D_copy == inf] = 0
    C = D_copy.astype(int)
    for match in m:
        paths = zuiduanlu[match]
        for i1 in range(len(paths) - 1):
            C[paths[i1]][paths[i1 + 1]] += 1
    print(C)
    euler(C, 1)

[[0 0 0 3 0]
 [1 0 1 0 0]
 [1 0 0 1 0]
 [0 0 0 0 4]
 [1 2 1 0 0]]
[1, 0, 3, 4, 0, 3, 4, 1]
[[0 0 0 1 0]
 [0 0 1 0 0]
 [1 0 0 1 0]
 [0 0 0 0 2]
 [0 1 1 0 0]]
[0, 3, 4, 0, 3, 4, 1, 3, 4, 1, 2, 0]
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 1 0 0]]
[2, 0, 3, 4, 0, 3, 4, 1, 3, 4, 1, 3, 4, 2]
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
